In [57]:
import os 
import pandas as pd
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

from nltk.corpus import stopwords

In [58]:
data = pd.read_csv("../02-Movie-reviews/reviews.csv")
data["clean_reviews"] = data.reviews.str.replace(r"[^A-Za-z]", " ").str.lower().str.split().apply(" ".join)
stop_words = stopwords.words("english")

<ipython-input-58-db55d2964ed7>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data["clean_reviews"] = data.reviews.str.replace(r"[^A-Za-z]", " ").str.lower().str.split().apply(" ".join)


In [59]:
load_dotenv()
cog_endpoint = os.getenv("COG_SERVICE_ENDPOINT")
cog_key = os.getenv("COG_SERVICE_KEY")

In [60]:
credential = AzureKeyCredential(cog_key)

In [61]:
client = TextAnalyticsClient(cog_endpoint, credential)

In [62]:
data = data[(data.clean_reviews.str.len()) < 5120]

In [63]:
def analyze_sentiment_df(string: str):
    return client.analyze_sentiment([string])[0].sentiment

data["pred"] = data.clean_reviews.apply(analyze_sentiment_df)

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(data.target, data.pred.map({"negative": "neg", "positive": "pos"}))

0.6501457725947521